In [5]:
import pandas as pd


In [8]:
df=pd.read_csv('your_output_file.csv')
df.head()


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race,Education,latitude,longitude
0,6/7/2017 11:33:27,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000,...,1,0,0,0,0,0,NaN,NaN,37.486324,-122.232523
1,6/10/2017 17:11:29,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0,...,2,0,0,0,0,0,NaN,NaN,37.779259,-122.419329
2,6/11/2017 14:53:57,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000,...,3,0,0,0,0,0,NaN,NaN,47.603832,-122.330062
3,6/17/2017 0:23:14,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000,...,7,0,0,0,0,0,NaN,NaN,37.368830,-122.036349
4,6/20/2017 10:58:51,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0,...,9,0,0,0,0,0,NaN,NaN,37.389389,-122.083210


In [12]:
import pandas as pd
import dash
from dash import dcc, html, dash_table
import altair as alt
from vega_datasets import data
from dash.dependencies import Input, Output

# Ensure timestamp is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Ensure latitude and longitude are numeric
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# Get min and max timestamps for range slider
min_date = df['timestamp'].min()
max_date = df['timestamp'].max()

# Convert timestamps to numerical values for the range slider
df['timestamp_numeric'] = (df['timestamp'] - min_date).dt.days

# Load world map data
world_map = alt.topo_feature(data.world_110m.url, "countries")

# Initialize Dash app
# app = dash.Dash(__name__)

# def world_map_layout():
#     return html.Div([
#         html.H1("Salary Distribution Dashboard", style={'textAlign': 'center'}),

#         # Timestamp Range Slider
#         html.Label("Select Date Range:"),
#         dcc.RangeSlider(
#             id="timestamp-slider",
#             min=0,
#             max=(max_date - min_date).days,
#             value=[0, (max_date - min_date).days],  # Default to full range
#             marks={i: (min_date + pd.Timedelta(days=i)).strftime('%Y-%m-%d') for i in range(0, (max_date - min_date).days + 1, (max_date - min_date).days // 10)},
#             step=1
#         ),

#         # Altair Chart using Iframe
#         html.Iframe(
#             id="altair-map",
#             style={"width": "100%", "height": "600px", "border": "none"}
#         ),

#         # Data Table
#         html.H3("Location-wise Salary Data", style={'textAlign': 'center'}),
#         dash_table.DataTable(
#             id="salary-table",
#             columns=[
#                 {"name": "Location", "id": "location"},
#                 {"name": "Average Salary", "id": "avg_salary", "type": "numeric", "format": {"specifier": "$,.2f"}},
#             ],
#             style_table={'overflowX': 'auto'},
#             page_size=10
#         )
#     ])

# app.layout = html.Div([
#     html.H1("Salary Distribution Dashboard", style={'textAlign': 'center'}),

#     # Timestamp Range Slider
#     html.Label("Select Date Range:"),
#     dcc.RangeSlider(
#         id="timestamp-slider",
#         min=0,
#         max=(max_date - min_date).days,
#         value=[0, (max_date - min_date).days],  # Default to full range
#         marks={i: (min_date + pd.Timedelta(days=i)).strftime('%Y-%m-%d') for i in range(0, (max_date - min_date).days + 1, (max_date - min_date).days // 10)},
#         step=1
#     ),

#     # Altair Chart using Iframe
#     html.Iframe(
#         id="altair-map",
#         style={"width": "100%", "height": "600px", "border": "none"}
#     ),

#     # Data Table
#     html.H3("Location-wise Salary Data", style={'textAlign': 'center'}),
#     dash_table.DataTable(
#         id="salary-table",
#         columns=[
#             {"name": "Location", "id": "location"},
#             {"name": "Average Salary", "id": "avg_salary", "type": "numeric", "format": {"specifier": "$,.2f"}},
#         ],
#         style_table={'overflowX': 'auto'},
#         page_size=10
#     )
# ])

# Callback to update data based on selected date range
@app.callback(
    [Output("altair-map", "srcDoc"), Output("salary-table", "data")],
    [Input("timestamp-slider", "value")]
)
def update_dashboard(selected_range):
    # Convert slider values back to timestamps
    start_date = min_date + pd.Timedelta(days=selected_range[0])
    end_date = min_date + pd.Timedelta(days=selected_range[1])

    # Filter data based on selected date range
    filtered_df = df[(df['timestamp'] >= start_date) & (df['timestamp'] <= end_date)]

    # Group by location after filtering
    grouped = filtered_df.groupby(['latitude', 'longitude'], as_index=False).agg(
        avg_salary=('totalyearlycompensation', 'mean'),
        location=('location', 'first')  # Keep one location name
    )

    # Create Altair map
    def create_altair_map():
        base = alt.Chart(world_map).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).properties(
            width=1200,
            height=750
        ).project("equirectangular")

        points = alt.Chart(grouped).mark_circle().encode(
            longitude="longitude:Q",
            latitude="latitude:Q",
            size=alt.Size("avg_salary:Q", scale=alt.Scale(range=[10, 250]), title="Average Salary"),
            color=alt.Color("avg_salary:Q", scale=alt.Scale(scheme="redyellowblue"), title="Average Salary"),
            tooltip=["location:N", "avg_salary:Q"]
        )

        return (base + points).interactive()

    # Save Altair chart as an HTML file
    altair_chart = create_altair_map()
    altair_chart.save("altair_map.html")

    return open("altair_map.html", "r").read(), grouped.to_dict("records")

# Run the dashboard
# if __name__ == "__main__":
#     app.run_server(debug=True)
